In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Automate

/content/drive/MyDrive/Automate


In [3]:
!pip install wget pathos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 132 kB 11.0 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=bbb815beb0d52fa411a75a177403595119f7286631d7b9351452535d7a32d9c6
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [4]:
!pip install nmslib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.4 MB 5.1 MB/s 
     |████████████████████████████████| 188 kB 51.5 MB/s 


In [5]:
!pip install bcolz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.2 MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp38-cp38-linux_x86_64.whl size=2876362 sha256=3f8ef75a3bac98c3d5db5a23bc5e35efbd595dfb615c14d31da6d6d44fa0f3c0
  Stored in directory: /root/.cache/pip/wheels/c8/f3/3b/5cf5f3997e4729700f17eb4faa8eb7dde1a209cdea66371b7a
Successfully built bcolz


In [6]:
!pip install pandas_summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 127 kB 5.4 MB/s 


In [7]:
!pip install isoweek

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install sentencepiece!=0.1.90,!=0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.2 MB/s 


In [ ]:
%cd /content/drive/MyDrive/Automate

/content/drive/MyDrive/Automate


In [ ]:
use_cache = False

In [9]:
from general_utils import get_step2_prerequisite_files



# Build Language Model From Docstrings

The goal is to build a language model using the docstrings, and use that language model to generate an embedding for each docstring.  

In [10]:
import torch,cv2
from lang_model_utils import lm_vocab, load_lm_vocab, train_lang_model
from general_utils import save_file_pickle, load_file_pickle
import logging
from pathlib import Path
from fastai.text.all import *
import pandas as pd

source_path = Path('./data/processed_data/')

with open(source_path/'train.docstring', 'r') as f:
    trn_data = f.readlines()

with open(source_path/'valid.docstring', 'r') as f:
    val_data = f.readlines()
    
with open(source_path/'test.docstring', 'r') as f:
    test_data = f.readlines()

df_data=trn_data+val_data
trn_raw=pd.DataFrame(trn_data)
val_raw=pd.DataFrame(val_data)
df_all = pd.concat([trn_raw, val_raw])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Preview what the raw data looks like: here are 10 docstrings

In [11]:
trn_raw[:10]

,0
0,docstring_tokens\n
1,"""fetch the i d of all docs of type """" doc_type """" from """" index""\n"
2,format the stats for each term into a compact tuple\n
3,"""returns term vectors for specified batch of docids for each docid yields the format : ( docid1 , { term1 : ( tf , df ) , term2 : ( tf , df ) ... } , docid2 )""\n"
4,"""returns term vectors for corpus one doc at a time for each docid yields the format : ( docid1 , { term1 : ( tf , df ) , term2 : ( tf , df ) ... } )""\n"
5,global application exception handler\n
6,called as the main body of the utility\n
7,add another validator .\n
8,join the specified validators .\n
9,wrap function and set description attr for nosetests to display .\n


## Pre-process data for language model

We will use the class  `build_lm_vocab` to prepare our data for the language model

In [12]:
vocab = lm_vocab(max_vocab=50000,
                 min_freq=10)

# fit the transform on the training data, then transform
all_flat_idx = vocab.fit_transform_flattened(df_data)

Look at the transformed data

In [13]:
all_flat_idx[:10]

array([  2,  46,  43,  10, 633,   2, 333,  45, 552,   2])

In [14]:
[vocab.itos[x] for x in all_flat_idx[:10]]

['_xbos_',
 'get',
 'data',
 'for',
 'storage',
 '_xbos_',
 'print',
 'function',
 '!',
 '_xbos_']

In [15]:
# apply transform to validation data
val_flat_idx = vocab.transform_flattened(val_data)

Save files for later use

In [16]:

vocab.save('./data/lang_model/vocab_v2.cls')
save_file_pickle('./data/lang_model/trn_flat_idx_list.pkl_v2', all_flat_idx)
save_file_pickle('./data/lang_model/val_flat_idx_list.pkl_v2', val_flat_idx)

## Train Fast.AI Language Model

This model will read in files that were created and train a [fast.ai](https://github.com/fastai/fastai/tree/master/fastai) language model.  This model learns to predict the next word in the sentence using fast.ai's implementation of [AWD LSTM](https://github.com/salesforce/awd-lstm-lm).  

The goal of training this model is to build a general purpose feature extractor for text that can be used in downstream models.  In this case, we will utilize this model to produce embeddings for function docstrings.

In [17]:
#Create data loaders
splits = [list(range_of(trn_raw)), list(range(len(val_raw), len(df_all)))]
tfms = [ attrgetter("text"),Tokenizer.from_df(0), Numericalize()]
dsets = Datasets(df_all, [tfms], splits=splits, dl_type=LMDataLoader)

In [18]:
bs,sl = 104,72
dls = dsets.dataloaders(bs=bs, seq_len=sl)

In [19]:
dls.show_batch(max_n=3)

,text,text_
0,xxbos open the item at index . xxbos takes a screen capture and saves it into the output file . xxbos adds a timer to the queue . xxbos returns help string of script xxbos configuration is done through the pool xxbos transform data read from e.s to python object : param serialized : result from e.s ( string ) : return : instance or instances of xxunk xxbos public method to,open the item at index . xxbos takes a screen capture and saves it into the output file . xxbos adds a timer to the queue . xxbos returns help string of script xxbos configuration is done through the pool xxbos transform data read from e.s to python object : param serialized : result from e.s ( string ) : return : instance or instances of xxunk xxbos public method to add
1,", this will be a general access page "" xxbos destroy a box xxbos declare a named queue . xxbos "" compute offset of reads sharing an interesting k - mer , opposite orientation . "" xxbos set a minimum call count allowance xxbos creates a connection that encapsulates weighted connection of input feature maps . xxbos create a new shell based on a already existed shell : param str source :","this will be a general access page "" xxbos destroy a box xxbos declare a named queue . xxbos "" compute offset of reads sharing an interesting k - mer , opposite orientation . "" xxbos set a minimum call count allowance xxbos creates a connection that encapsulates weighted connection of input feature maps . xxbos create a new shell based on a already existed shell : param str source : the"
2,"of an output file with the specified name . assumes that the output file name is specified in the command line args . xxbos coil a gcode linear move xxunk . xxbos "" add unique indexes for models which have a xxunk attribute . these must be defined as partially unique indexes , which django does not support . the unique indexes are defined so that no two * current * versions","an output file with the specified name . assumes that the output file name is specified in the command line args . xxbos coil a gcode linear move xxunk . xxbos "" add unique indexes for models which have a xxunk attribute . these must be defined as partially unique indexes , which django does not support . the unique indexes are defined so that no two * current * versions can"


In [20]:
# create lang model data
config = awd_lstm_lm_config.copy()
config.update({'input_p': 0.6, 'output_p': 0.4, 'weight_p': 0.5, 'embed_p': 0.1, 'hidden_p': 0.2})
model = get_language_model(AWD_LSTM, len(dls.vocab), config=config)

In [21]:
opt_func = partial(Adam, wd=0.1, eps=1e-7)
cbs = [MixedPrecision(), GradientClip(0.1)] + rnn_cbs(alpha=2, beta=1)

In [22]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), opt_func=opt_func, cbs=cbs, metrics=[accuracy, Perplexity()])

In [23]:
learn.fit_one_cycle(10, 5e-3, moms=(0.8,0.7,0.8), div=10)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.166731,4.883075,0.262241,132.036087,06:56
1,4.666011,4.398644,0.308935,81.340492,06:53
2,4.492945,4.201508,0.325850,66.786972,06:56
3,4.386054,4.080843,0.334678,59.195351,06:51
4,4.313473,3.999540,0.342055,54.573032,06:52
5,4.230206,3.932603,0.348146,51.039642,06:51
6,4.153339,3.862401,0.354500,47.579464,06:50
7,4.103649,3.822985,0.358518,45.740547,06:47
8,4.082349,3.798474,0.361391,44.633038,06:50
9,4.052903,3.792332,0.361939,44.359745,06:51


Save language model and learner

In [24]:
learn.save('lang_model_learner_v2.fai')
lang_model_new = learn.model.eval()
torch.save(lang_model_new, './data/lang_model/lang_model_gpu_v2.torch')
torch.save(lang_model_new.cpu(), './data/lang_model/lang_model_cpu_v2.torch')

# Load Model and Encode All Docstrings

Now that we have trained the language model, the next step is to use the language model to encode all of the docstrings into a vector. 

In [11]:
from lang_model_utils import load_lm_vocab
vocab = load_lm_vocab('./data/lang_model/vocab_v2.cls')
idx_docs = vocab.transform(trn_data + val_data, max_seq_len=30, padding=False)
lang_model = torch.load('./data/lang_model/lang_model_gpu_v2.torch', 
                        map_location=lambda storage, loc: storage)

In [12]:
lang_model.eval()

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(28128, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(28128, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=28128, bias=True)
    (output_dp): RNNDropout()
  )
)

**Note:** the below code extracts embeddings for docstrings one docstring at a time, which is very inefficient.  Ideally, you want to extract embeddings in batch but account for the fact that you will have padding, etc. when extracting the hidden states.  For this tutorial, we only provide this minimal example, however you are welcome to improve upon this and sumbit a PR!

In [13]:
from tqdm import *

def list2arr(l):
    "Convert list into pytorch Variable."
    return torch.Tensor(np.expand_dims(np.array(l), -1)).long().cpu()

def make_prediction_from_list(model, l):
    """
    Encode a list of integers that represent a sequence of tokens.  The
    purpose is to encode a sentence or phrase.

    Parameters
    -----------
    model : fastai language model
    l : list
        list of integers, representing a sequence of tokens that you want to encode

    """
    arr = list2arr(l)# turn list into pytorch Variable with bs=1
    model.reset()  # language model is stateful, so you must reset upon each prediction
    hidden_states = model(arr)[-1][-1] # RNN Hidden Layer output is last output, and only need the last layer

    #return avg-pooling, max-pooling, and last hidden state
    return hidden_states.mean(0), hidden_states.max(0)[0], hidden_states[-1]


def get_embeddings(lm_model, list_list_int):
    """
    Vectorize a list of sequences List[List[int]] using a fast.ai language model.

    Paramters
    ---------
    lm_model : fastai language model
    list_list_int : List[List[int]]
        A list of sequences to encode

    Returns
    -------
    tuple: (avg, mean, last)
        A tuple that returns the average-pooling, max-pooling over time steps as well as the last time step.
    """
    n_rows = len(list_list_int)
    n_dim = lm_model[0].emb_sz
    avgarr = np.empty((n_rows, n_dim))
    maxarr = np.empty((n_rows, n_dim))
    lastarr = np.empty((n_rows, n_dim))

    for i in tqdm_notebook(range(len(list_list_int))):
        avg_, max_, last_ = make_prediction_from_list(lm_model, list_list_int[i])
        avgarr[i,:] = avg_.data.numpy()
        maxarr[i,:] = max_.data.numpy()
        lastarr[i,:] = last_.data.numpy()

    return avgarr, maxarr, lastarr

In [14]:
%%time
avg_hs, max_hs, last_hs = get_embeddings(lang_model, idx_docs)

<ipython-input-13-e02585c9bc19>:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(list_list_int))):


  0%|          | 0/267630 [00:00<?, ?it/s]

CPU times: user 4h 47min 17s, sys: 5min 50s, total: 4h 53min 7s
Wall time: 4h 54min 7s


### Do the same thing for the test set

In [16]:
idx_docs_test = vocab.transform(test_data, max_seq_len=30, padding=False)
avg_hs_test, max_hs_test, last_hs_test = get_embeddings(lang_model, idx_docs_test)

<ipython-input-13-e02585c9bc19>:48: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(list_list_int))):


  0%|          | 0/40794 [00:00<?, ?it/s]

# Save Language Model Embeddings For Docstrings

In [15]:
savepath = Path('./data/lang_model/')
np.save(savepath/'avg_emb_dim500_v2.npy', avg_hs)
np.save(savepath/'max_emb_dim500_v2.npy', max_hs)
np.save(savepath/'last_emb_dim500_v2.npy', last_hs)

In [17]:
# save the test set embeddings also
np.save(savepath/'avg_emb_dim500_test_v2.npy', avg_hs_test)
np.save(savepath/'max_emb_dim500_test_v2.npy', max_hs_test)
np.save(savepath/'last_emb_dim500_test_v2.npy', last_hs_test)

### Create search index using `nmslib` 

[nmslib](https://github.com/nmslib/nmslib) is a great library for doing nearest neighbor lookups, which we will use as a search engine for finding nearest neighbors of comments in vector-space.  

The convenience function `create_nmslib_search_index` builds this search index given a matrix of vectors as input.



In [18]:
from general_utils import create_nmslib_search_index
import nmslib
from lang_model_utils import Query2Emb
from pathlib import Path
import numpy as np
from lang_model_utils import load_lm_vocab
import torch

In [19]:
# Load matrix of vectors
loadpath = Path('./data/lang_model/')
avg_emb_dim500 = np.load(loadpath/'avg_emb_dim500_test_v2.npy')

In [20]:
# Build search index (takes about an hour on a p3.8xlarge)
dim500_avg_searchindex = create_nmslib_search_index(avg_emb_dim500)

In [21]:
# save search index
dim500_avg_searchindex.saveIndex('./data/lang_model/dim500_avg_searchindex.nmslib')

After building this search index with nmslib, we can do fast nearest-neighbor lookups.  We use the `Query2Emb` object to help convert strings to the embeddings: 

In [22]:
dim500_avg_searchindex = nmslib.init(method='hnsw', space='cosinesimil')
dim500_avg_searchindex.loadIndex('./data/lang_model/dim500_avg_searchindex.nmslib')

In [23]:
lang_model = torch.load('./data/lang_model/lang_model_cpu_v2.torch',map_location=torch.device('cpu'))
vocab = load_lm_vocab('./data/lang_model/vocab_v2.cls')

q2emb = Query2Emb(lang_model = lang_model.cpu(),
                  vocab = vocab)

The method `Query2Emb.emb_mean` will allow us to use the langauge model we trained earlier to generate a sentence embedding given a string.   Here is an example, `emb_mean` will return a numpy array of size (1, 500).

In [24]:
query = q2emb.emb_mean('Read data into pandas dataframe')
query.shape

(400,)

**Make search engine to inspect semantic similarity of phrases**.  This will take 3 inputs:

1. `nmslib_index` - this is the search index we built above.  This object takes a vector and will return the index of the closest vector(s) according to cosine distance.  
2. `ref_data` - this is the data for which the index refer to, in this case will be the docstrings. 
3. `query2emb_func` - this is a function that will convert a string into an embedding.

In [25]:
class search_engine:
    def __init__(self, 
                 nmslib_index, 
                 ref_data, 
                 query2emb_func):
        
        self.search_index = nmslib_index
        self.data = ref_data
        self.query2emb_func = query2emb_func
    
    def search(self, str_search, k=3):
        query = self.query2emb_func(str_search)
        idxs, dists = self.search_index.knnQuery(query, k=k)
        
        for idx, dist in zip(idxs, dists):
            print(f'cosine dist:{dist:.4f}\n---------------\n', self.data[idx])

In [26]:
se = search_engine(nmslib_index=dim500_avg_searchindex,
                   ref_data = test_data,
                   query2emb_func = q2emb.emb_mean)

## Manually Inspect Phrase Similarity

Compare a user-supplied query vs. vectorized docstrings on test set.  We can see that similar phrases are not exactly the same, but the nearest neighbors are reasonable.  

In [27]:
import logging
logging.getLogger().setLevel(logging.ERROR)

In [28]:
se.search('read csv into pandas dataframe')

cosine dist:0.0000
---------------
 parallel function to create shapely points from latitude / longitude pair in dataframe

cosine dist:0.0000
---------------
 parse xfoil polars and concatente data in dataframe

cosine dist:0.0000
---------------
 reads a file into a pandas dataframe



In [29]:
se.search('django files upload')

cosine dist:0.6539
---------------
 return the angle between the two points in range from -180 to 180 .

cosine dist:0.6539
---------------
 database setup before the tests .

cosine dist:0.6539
---------------
 runs before each test .



In [30]:
se.search('read data from source and target files and put into buckets .')

cosine dist:0.0000
---------------
 return the angle between the two points in range from -180 to 180 .

cosine dist:0.0000
---------------
 database setup before the tests .

cosine dist:0.0000
---------------
 runs before each test .



In [31]:
se.search('validate data using model level methods .')

cosine dist:0.0000
---------------
 return the angle between the two points in range from -180 to 180 .

cosine dist:0.0000
---------------
 database setup before the tests .

cosine dist:0.0000
---------------
 runs before each test .



In [32]:
se.search('send out email notification')

cosine dist:0.0000
---------------
 route a notification

cosine dist:0.0000
---------------
 add a new observer which is notified when this class fires a notification

cosine dist:0.0000
---------------
 shows a system tray notification



In [33]:
se.search('save pickle file')

cosine dist:0.6705
---------------
 """ this method is used to convert manual reminder object to json format ."

cosine dist:0.6705
---------------
 "update pair at index w/ key , value & backup - encrypted passwd ."

cosine dist:0.6705
---------------
 "given a ' config ' dictionary , instantiate this object in such a way that it will produce the field object described by the dictionary ."



In [34]:
se.search('rest api')

cosine dist:0.0000
---------------
 process metric by sending it to datadog api

cosine dist:0.0000
---------------
 post a request to rest query api

cosine dist:0.0000
---------------
 runs the rest api

